# Real Estate Regression
### Building a Regression Model to Predict Housing Prices in NY City 
### Jackson Patrick & Trevor Chartier

## Part 1: Data Processing

In [ ]:
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree
from geopy.distance import geodesic # may need to run pip install geopy
import pandas.api.types as pd_types

Read In Dataset

In [ ]:

house_df = pd.read_csv("data/NY-House-Dataset.csv")
print(house_df.shape)
house_df.head()

Removing Features

TODO: 

** This is where we will add in some analsysis of categorical columns like BROKERTITLE or Address to see if we even want to keep them


In [ ]:
# Some code to get you started 
len(house_df["STREET_NAME"].unique())

len(house_df["BROKERTITLE"].unique())

In [ ]:
# It appears that in this dataset, missing values are filled in with the mean value for the column
# We want to identify columns with potentially large numbers of missing values

def num_filled(colName):
    mean_val = np.mean(house_df[colName])
    is_close = np.isclose(house_df[colName], mean_val, atol=1e-9)
    return sum(is_close)

for column in house_df:
    if(pd_types.is_numeric_dtype(house_df[column])):
        print(num_filled(column), "instance in", column, " have been filled with the mean value of the column")

### Feature Engineering: Distance to Nearest Subway Station

In [ ]:
subway_df = pd.read_csv("data/MTA_Subway_Stations.csv")
subway_df = subway_df[["GTFS Latitude", "GTFS Longitude"]]
subway_df.rename(columns={"GTFS Latitude": "Latitude", "GTFS Longitude":"Longitude"}, inplace=True)
print(subway_df.shape[0], " Subway Stations Included in NYC Dataset")
subway_df.head()

In [ ]:
subway_coords = subway_df[['Latitude','Longitude']].values
subway_tree = cKDTree(subway_coords)

subway_distances = []

for _, property in house_df.iterrows():
    property_coord = (property["LATITUDE"], property["LONGITUDE"])
    
    distance, idx = subway_tree.query([property["LATITUDE"], property["LONGITUDE"]])
    
    nearest_subway_coord = subway_coords[idx]
    precise_distance = np.round(geodesic(property_coord, nearest_subway_coord).miles,6)
    subway_distances.append(precise_distance)
    
house_df["NEAREST_SUB_STATION"] = subway_distances

In [ ]:
house_df.head()